## Dev env configuration

In [1]:
# install the required python packages
# !pip install numpy pandas

## Imports

In [19]:
# import the necessary packages
from typing import Optional
from typing import Union
from typing import List
from dataclasses import dataclass

## Implementing a simple risk/reward calculator

In [11]:
@dataclass(frozen=True)
class TradeRiskParams:
    # define the data schema for risk/reward parameters for a given trade
    amount_to_risk: float
    percent_risk: float
    shares_to_buy: float
    total_investment: float

In [12]:
@dataclass(frozen=True)
class RLevelOutput:
    # define the data schema for R-level outputs
    r_level: int
    potential_pl: float
    price: float

In [51]:
class RiskRewardCalculator:

    # define the list of default R-values to compute
    DEFAULT_R_VALUES = [1, 2, 3, 4, 5]

    def __init__(
            self,
            total_account_value: float,
            buy_point: float,
            stop_loss: float,
            risk_rate: float = 0.01
    ) -> None:
        # store the total account value, buy point, stop loss, and risk rate
        self.total_account_value = total_account_value
        self.buy_point = buy_point
        self.stop_loss = stop_loss
        self.risk_rate = risk_rate

        # calculate the amount to risk on each trade
        self.amount_to_risk = self.total_account_value * self.risk_rate

        # determine the price difference between the buy point and stop loss,
        # then derive the number of shares to buy based on the amount to risk
        self.price_delta = self.buy_point - self.stop_loss
        self.shares_to_buy = self.amount_to_risk / self.price_delta

        # calculate the total amount of money to be invested based on the
        # number of shares to buy and the buy point
        self.total_investment = self.shares_to_buy * self.buy_point

        # derive the overall risk percentage of the account
        amount_ratio = self.amount_to_risk / self.total_account_value
        self.risk_pct_of_account = amount_ratio * 100

    def get_risk_parameters(self) -> TradeRiskParams:
        # construct and return the calculated risk parameters
        return TradeRiskParams(
            amount_to_risk=self.amount_to_risk,
            percent_risk=self.risk_pct_of_account,
            shares_to_buy=self.shares_to_buy,
            total_investment=self.total_investment
        )

    def r_levels(
            self,
            r_levels: Optional[List[Union[int, float]]] = None
    ) -> List[RLevelOutput]:
        # check to see if risk/reward levels were not provided
        if not isinstance(r_levels, list):
            # use the default risk/reward levels
            r_levels = self.DEFAULT_R_VALUES

        # initialize the list of R-level outputs
        results = []

        # loop over the levels
        for r in r_levels:
            # calculate (1) the potential profit/loss, and (2) the price at
            # the current level
            potential_pl = r * self.amount_to_risk
            price = self.buy_point + (r * self.price_delta)

            # update the results list
            results.append(RLevelOutput(
                r_level=r,
                potential_pl=potential_pl,
                price=price
            ))

        # return the list of r-level results
        return results

    def r_level_at_price(self, price: float) -> float:
        # compute and return the r-level for the input price
        return (price - self.buy_point) / self.price_delta

## Using our risk/reward calculator in real-world trading situations

In [52]:
calc = RiskRewardCalculator(
    total_account_value=25_000,
    buy_point=57.70,
    stop_loss=55
)

In [53]:
calc.get_risk_parameters()

TradeRiskParams(amount_to_risk=250.0, percent_risk=1.0, shares_to_buy=92.5925925925925, total_investment=5342.592592592588)

In [54]:
calc.r_levels()

[RLevelOutput(r_level=1, potential_pl=250.0, price=60.400000000000006),
 RLevelOutput(r_level=2, potential_pl=500.0, price=63.10000000000001),
 RLevelOutput(r_level=3, potential_pl=750.0, price=65.80000000000001),
 RLevelOutput(r_level=4, potential_pl=1000.0, price=68.50000000000001),
 RLevelOutput(r_level=5, potential_pl=1250.0, price=71.20000000000002)]

In [55]:
calc.r_level_at_price(71.2)

4.999999999999995